## 1. Import Libraries

In [6]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam,SGD
from tensorflow.keras.utils import to_categorical
import numpy as np

## 2. Load MNIST Dataset

In [7]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images = train_images / 255.0
test_images = test_images / 255.0

# Convert labels to one-hot encoding
train_labels_one_hot = to_categorical(train_labels)
test_labels_one_hot = to_categorical(test_labels)

## 3. Create Model

In [8]:
model = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(10, activation='relu'),
    Dropout(0.3),
    Dense(10, activation='sigmoid')  # Use sigmoid in the output layer
])

adam=Adam(learning_rate=0.001)
model.compile(optimizer=adam,
              loss='categorical_crossentropy',  # Use categorical_crossentropy for one-hot labels
              metrics=['accuracy'])

/home/reza/.local/lib/python3.12/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


## 4. Train Model

In [10]:
history = model.fit(train_images, train_labels_one_hot, epochs=15, batch_size=32, validation_data=(test_images, test_labels_one_hot))

# Evaluate the model
loss, accuracy = model.evaluate(test_images, test_labels_one_hot)
print(f"Test accuracy: {accuracy * 100:.2f}%")

weights = model.get_weights()

Epoch 1/15
 159/1875 ━━━━━━━━━━━━━━━━━━━━ 1s 967us/step - accuracy: 0.7895 - loss: 0.6296

2024-10-25 15:04:26.939165: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 188160000 exceeds 10% of free system memory.


1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.7843 - loss: 0.6390 - val_accuracy: 0.9131 - val_loss: 0.3190
Epoch 2/15
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.7831 - loss: 0.6455 - val_accuracy: 0.9151 - val_loss: 0.3092
Epoch 3/15
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.7833 - loss: 0.6466 - val_accuracy: 0.9136 - val_loss: 0.3223
Epoch 4/15
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.7827 - loss: 0.6383 - val_accuracy: 0.9132 - val_loss: 0.3123
Epoch 5/15
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.7841 - loss: 0.6367 - val_accuracy: 0.9128 - val_loss: 0.3151
Epoch 6/15
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.7826 - loss: 0.6386 - val_accuracy: 0.9156 - val_loss: 0.3141
Epoch 7/15
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.7856 - loss: 0.6367 - val_accuracy: 0.9142 - val_loss: 0.3078
Epoch 8/15
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.7825 - loss: 0.6389 - val_accurac

## 5. Utils Function

### 5.1. Save Raw Weights

In [11]:
def save_model_weights_to_file(model, filename):
    with open(filename, 'w') as file:
        for layer_idx, layer in enumerate(model.layers):
            # Check if the layer has weights (e.g., Dense layers)
            if(len(layer.get_weights()) > 0):
                weights, biases = layer.get_weights()

                # Iterate over the weights matrix
                for i in range(weights.shape[0]):  # Iterate over neurons in the current layer
                    for j in range(weights.shape[1]):  # Iterate over neurons in the next layer
                        weight_value = weights[i, j]
                        # Write the weight information to the file
                        file.write(f"In layer {layer_idx}, neuron {i} is connected to neuron {j} in the next layer with weight value: {weight_value}\n")


save_model_weights_to_file(model, 'model_weights.txt')

### 5.2. Extract All Weights

In [12]:
def extract_weights(model, filename):
    with open(filename, 'w') as file:
        for layer_idx, layer in enumerate(model.layers):
            # Check if the layer has weights (e.g., Dense layers)
            if(len(layer.get_weights()) > 0):
                weights, biases = layer.get_weights()

                # Iterate over the weights matrix
                for i in range(weights.shape[0]):  # Iterate over neurons in the current layer
                    for j in range(weights.shape[1]):  # Iterate over neurons in the next layer
                        weight_value = weights[i, j]
                        # Write the weight information to the file
                        file.write(str(weight_value)+"\n")


extract_weights(model, 'pure_weights.txt')

## 6. Select Random Input and Their Predict and Trure Label for Test in Inference Phase on FPGA

In [13]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

test_images = test_images / 255.0

np.random.seed(0)
random_idxs = np.random.choice(test_images.shape[0], 200, replace=False)
selected_images = test_images[random_idxs]
selected_labels = test_labels[random_idxs]

selected_images = np.expand_dims(selected_images, axis=-1) 

predictions = model.predict(selected_images)

predicted_classes = np.argmax(predictions, axis=1)

for i, (pred, true) in enumerate(zip(predicted_classes, selected_labels)):
  print(f"Image {i+1}: Predicted class = {pred}, True class = {true}"+'\n')

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
Image 1: Predicted class = 8, True class = 8

Image 2: Predicted class = 8, True class = 7

Image 3: Predicted class = 1, True class = 1

Image 4: Predicted class = 9, True class = 7

Image 5: Predicted class = 1, True class = 1

Image 6: Predicted class = 2, True class = 2

Image 7: Predicted class = 1, True class = 1

Image 8: Predicted class = 9, True class = 9

Image 9: Predicted class = 0, True class = 5

Image 10: Predicted class = 4, True class = 4

Image 11: Predicted class = 5, True class = 5

Image 12: Predicted class = 6, True class = 6

Image 13: Predicted class = 3, True class = 3

Image 14: Predicted class = 2, True class = 2

Image 15: Predicted class = 0, True class = 0

Image 16: Predicted class = 7, True class = 7

Image 17: Predicted class = 6, True class = 6

Image 18: Predicted class = 4, True class = 4

Image 19: Predicted class = 6, True class = 6

Image 20: Predicted class = 4, True class = 4

Image 21: Predicted class = 3, 